In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torchtext

## Data

In [0]:
TEXT = torchtext.data.Field(tokenize='spacy',
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
LABEL = torchtext.data.LabelField(dtype=torch.float)

train_txt, test_txt = torchtext.datasets.IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(train_txt)
LABEL.build_vocab(train_txt)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
BATCH_SIZE = 32

train_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_txt, test_txt),
    batch_size = BATCH_SIZE,
    device = device
)

## Model

In [0]:
class Transformer(nn.Module):

  def __init__(self, num_token, embed_size, num_head, len_ff, output_size, num_layers, dropout=0.5):
    super(Transformer, self).__init__()
    self.model_type = 'Transformer'
    self.src_mask = None
    self.pos_encoder = PositionalEncoding(embed_size, dropout)
    encoder_layers = TransformerEncoderLayer(embed_size, num_head, len_ff, dropout)
    self.trans_encoder = TransformerEncoder(encoder_layers, num_layers)
    self.embed = nn.Embedding(num_token, embed_size)
    self.embed_size = embed_size
    self.decoder = nn.Linear(embed_size, output_size)
    self.sig = nn.Sigmoid()

    self.init_weights()

  def _generate_square_subsequent_mask(self, size):
    mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

  def init_weights(self):
    initrange = 0.1
    self.embed.weight.data.uniform_(-initrange, initrange)
    self.decoder.bias.data.zero_()
    self.decoder.weight.data.uniform_(-initrange, initrange)

  def forward(self, src):
    if self.src_mask is None or self.src_mask.size(0) != len(src):
        device = src.device
        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        self.src_mask = mask

    src = self.embed(src) * math.sqrt(self.embed_size)
    src = self.pos_encoder(src)
    output = self.trans_encoder(src, self.src_mask)
    output = output.sum(dim=0).float()
    output = self.decoder(output)
    output = self.sig(output)
    return output.squeeze()

In [0]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [0]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
out_size = 1
dropout = 0.2 # the dropout value
model = Transformer(ntokens, emsize, nhead, nhid, out_size, nlayers, dropout).to(device)

## Training

In [0]:
criterion = nn.BCELoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [0]:
import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    i = 0
    for batch in train_iterator:
        data, targets = batch.text, batch.label
        optimizer.zero_grad()
        output = model(data)
        print(data.shape, targets.shape)
        #loss = criterion(output.view(-1, ntokens), targets)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if i % log_interval == 0 and i > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            i += 32
            start_time = time.time()

In [0]:
def test(test_model, iterator):
    test_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for batch in iterator:
            data, targets = batch.data, batch.label
            output = test_model(data)
            #output_flat = output.view(-1, ntokens)
            total_loss += criterion(output, targets).item()
    return total_loss

In [0]:
best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()

    #val_loss = evaluate(model, val_data)
    #print('-' * 89)
    #print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
    #      'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
    #                                 val_loss, math.exp(val_loss)))
    #print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step()

torch.Size([1389, 32]) torch.Size([32])
torch.Size([452, 32]) torch.Size([32])
torch.Size([694, 32]) torch.Size([32])
torch.Size([1084, 32]) torch.Size([32])
torch.Size([870, 32]) torch.Size([32])
torch.Size([1121, 32]) torch.Size([32])
torch.Size([938, 32]) torch.Size([32])
torch.Size([1152, 32]) torch.Size([32])
torch.Size([908, 32]) torch.Size([32])
torch.Size([1165, 32]) torch.Size([32])
torch.Size([888, 32]) torch.Size([32])
torch.Size([1248, 32]) torch.Size([32])
torch.Size([1162, 32]) torch.Size([32])
torch.Size([952, 32]) torch.Size([32])
torch.Size([1145, 32]) torch.Size([32])
torch.Size([817, 32]) torch.Size([32])
torch.Size([1036, 32]) torch.Size([32])
torch.Size([827, 32]) torch.Size([32])
torch.Size([1014, 32]) torch.Size([32])
